#### Sprint 11 Project

In [1]:
import pandas as pd

In [2]:
try:
    insurance_df = pd.read_csv('/datasets/insurance_us.csv')

except:
    insurance_df = pd.read_csv('insurance_us.csv')

#### EDA/ Data Preprocessing

In [3]:
insurance_df.head()

,Gender,Age,Salary,Family members,Insurance benefits
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


In [4]:
insurance_df.rename(columns={"Gender": "gender", "Age": "age", "Salary": "salary", "Family members": "family_members", "Insurance benefits": "insurance_benefits"})

,gender,age,salary,family_members,insurance_benefits
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
...,...,...,...,...,...
4995,0,28,35700,2,0
4996,0,34,52400,1,0
4997,0,20,33900,2,0
4998,1,22,32700,3,0


In [5]:
insurance_df.isna().sum()

Gender                0
Age                   0
Salary                0
Family members        0
Insurance benefits    0
dtype: int64

In [6]:
insurance_df.duplicated().sum()

153

In [7]:
insurance_df.drop_duplicates(inplace=True)

insurance_df.duplicated().sum()

0

In [8]:
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Gender              4847 non-null   int64
 1   Age                 4847 non-null   int64
 2   Salary              4847 non-null   int64
 3   Family members      4847 non-null   int64
 4   Insurance benefits  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB
